# Monitoramento de Rotas e Latência com TraceRoute e Ping
## Redes de Computadores
### Alunos: Leonardo de Oliveira Nanes, Gabriel Lázaro e Gabriel Calabrese 

Nesta apresentação, vamos explorar o conceito de **TraceRoute** com **Ping** imbutido para monitoramento de redes e visualização de hops, latência e perda de pacotes entre o host e o destino. Além disso, plotaremos as rotas geograficamente em um mapa interativo.

---

### Estrutura da apresentação:
-  Introdução aos conceitos de TraceRoute e Ping.
-  Implementação prática das ferramentas.
-  Visualização das rotas no mapa.

---

## O que é TraceRoute?

O **TraceRoute** é uma ferramenta de diagnóstico de redes que rastreia o caminho percorrido pelos pacotes de dados entre a origem (host) e o destino. Ele identifica os roteadores intermediários (hops) que os pacotes atravessam até chegar ao seu destino final.

- **Funcionamento**: O TraceRoute envia pacotes com valores incrementais de TTL (Time to Live). Quando o TTL chega a 0, o roteador emite uma mensagem ICMP "time exceeded", revelando sua presença. Esse processo se repete até alcançar o destino final ou até o TTL máximo.

---

## O que é Ping?

O **Ping** é uma ferramenta usada para testar a conectividade entre dois dispositivos em uma rede. Ele mede a latência (tempo de resposta) e a perda de pacotes. O Ping utiliza pacotes ICMP "Echo Request" e "Echo Reply" para testar se um dispositivo está acessível e para obter informações sobre o tempo de resposta.

- **Latência**: Tempo que um pacote leva para ir até o destino e voltar.
- **Perda de Pacotes**: Proporção de pacotes enviados que não receberam resposta.


# IMPORTAÇÃO DE BIBLIOTECAS

In [1]:
import socket
import folium
import requests
import ipaddress
import pingparsing

## Funções: TraceRoute, Ping e Geolocalização

Aqui estão as funções principais para realizar o TraceRoute com Ping e para obter as coordenadas dos IPs através de uma API pública.

- `traceroute_with_ping`: Realiza o TraceRoute até o destino especificado, verificando a latência e perda de pacotes a cada hop.
- `get_coordinates`: Obtém as coordenadas geográficas do IP usando uma API para plotagem de roteadores no mapa.
- `is_private_ip`: Verifica se o IP é privado e, assim, não acessível por APIs públicas.


# TRACEROUTE COM PING

In [ ]:


def traceroute_with_ping(host, max_hops=30, timeout=2, ping_count=4):
    print(f"TraceRoute para {host}:")

    hops = []
    dest_addr = socket.gethostbyname(host)
    port = 33434  
    icmp = socket.getprotobyname('icmp')
    udp = socket.getprotobyname('udp')
    ping_parser = pingparsing.PingParsing()
    transmitter = pingparsing.PingTransmitter()

    for ttl in range(1, max_hops + 1):
        recv_socket = socket.socket(socket.AF_INET, socket.SOCK_RAW, icmp)
        send_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, udp)
        send_socket.setsockopt(socket.SOL_IP, socket.IP_TTL, ttl)
        recv_socket.settimeout(timeout)

        recv_socket.bind(("", port))
        send_socket.sendto(b"", (host, port))

        curr_addr = None
        try:
            _, curr_addr = recv_socket.recvfrom(512)
            curr_addr = curr_addr[0]
            hops.append(curr_addr)

            # Fazer ping no hop atual
            transmitter.destination = curr_addr
            transmitter.count = ping_count
            ping_result = transmitter.ping()

            if ping_result.returncode == 0:
                ping_stats = ping_parser.parse(ping_result.stdout)

                packet_loss = ping_stats.packet_loss_rate
                packets_sent = ping_stats.packet_transmit
                min_time = ping_stats.rtt_min
                avg_time = ping_stats.rtt_avg
                max_time = ping_stats.rtt_max
                mdev_time = ping_stats.rtt_mdev

                print(
                    f"{ttl}: {curr_addr} | Sent: {packets_sent} | Loss: {packet_loss} % | avg: {avg_time}ms | Best: {min_time}ms | Worst: {max_time}ms | Std.Dev: {mdev_time}")
            else:
                print(f"{ttl}: {curr_addr} | Erro no ping")

        except socket.timeout:
            print(f"{ttl}: * (timeout)")
        except socket.error as e:
            print(f"{ttl}: erro no socket - {e}")
        finally:
            send_socket.close()
            recv_socket.close()

        if curr_addr == dest_addr:
            print("Destino alcançado!")
            break
    return hops

# MAPA

## Vendo se o IP é privado

In [ ]:
def is_private_ip(ip):
    return ipaddress.ip_address(ip).is_private

## Pegando as coordenadas

In [ ]:
def get_coordinates(ip):
    if is_private_ip(ip):
        print(f"IP {ip} é privado. Sem coordenadas públicas conhecidas.")
        return None

    try:
        response = requests.get(f"http://ip-api.com/json/{ip}")
        if response.status_code == 200:
            data = response.json()
            if data['status'] == 'success':
                lat, lon = data['lat'], data['lon']
                # Exibe as coordenadas no console
                print(f"Coordenadas de {ip}: ({lat}, {lon})")
                return lat, lon
            else:
                print(f"API retornou falha para o IP {ip}: {data['message']}")
                return None
        else:
            print(
                f"Erro ao conectar-se à API para o IP {ip}. Status HTTP: {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Erro de conexão ao tentar obter localização de {ip}: {e}")
        return None

# PLOTANDO O MAPA

## Execução do Programa



In [ ]:
def main():
    destino = input("Digite o IP ou endereço do host: ")
    try:
        hops = traceroute_with_ping(destino)
        #if hops:
           # plot_route(hops)
       # else:
          #  print("Não foi possível rastrear a rota.")
    except socket.gaierror:
        print(f"Erro: Host '{destino}' não encontrado. Verifique o endereço e tente novamente.")
    except Exception as e:
        print(f"Erro inesperado: {e}")

main()
